In [1]:
using StatsPlots, Statistics, Distributions, Random, PyPlot

In [25]:
n_bins = 4
μ = 6
σ = 3

RV_list = Array{Distribution}(undef, 5)

for i = 1:5
    temp_RV = Normal(μ, σ)
    RV_list[i] = temp_RV
end


In [26]:
function generate_midpoints(n_bins)
    quantiles = Array{Float64}(undef, n_bins)

    #generate the quantile midpoints
    for i = 1:n_bins
        quantiles[i] = i/n_bins - (1/2)*(1/n_bins)
    end

    return vec(quantiles)
end

generate_midpoints (generic function with 1 method)

In [27]:
function add_distributions(list1, list2)
    sum_list = Array{Float64}(undef, length(list1)^2)
    
    list1 = repeat(list1, length(list1))
    list1 = sort(list1)
    list2 = repeat(list2, length(list2))
    
    for i in eachindex(sum_list)
        sum_list[i] = list1[i] + list2[i]
    end
    
    return vec(sort(sum_list))
end

add_distributions (generic function with 1 method)

In [28]:
function re_bin(distribution, n_bins)
    rebinned = reshape(distribution, n_bins, n_bins)
    rebinned = mean(rebinned, dims = 1)
    
    return vec(rebinned)
end

re_bin (generic function with 1 method)

In [29]:
quantile_boundaries = generate_midpoints(n_bins)
# println(quantile_boundaries)

final_RV = pop!(RV_list)
final_RV = quantile(final_RV, quantile_boundaries)
println(final_RV)

for RV in RV_list
    temp_list = quantile(RV, quantile_boundaries)
    
    final_RV = add_distributions(temp_list, final_RV)
    final_RV = re_bin(final_RV, n_bins)
    println(final_RV) 
end


[2.548951858871975, 5.044081908106874, 6.955918091893126, 9.451048141128025]
[7.447210300616687, 10.898258441744712, 13.101741558255288, 16.552789699383315]
[12.58444826533468, 16.58636718559035, 19.413632814409652, 23.415551734665318]
[17.859403924834584, 22.29270386273126, 25.70729613726874, 30.140596075165412]
[23.24220483874474, 28.03802694689146, 31.961973053108537, 36.75779516125526]


In [30]:
mean(final_RV)

30.0

In [31]:
mean(final_RV)/5

6.0